In [1]:
from tensorflow.keras.datasets import cifar10
import cv2
import numpy as np

# Load CIFAR-10
(x_train, _), (x_test, _) = cifar10.load_data()

# Convert to grayscale and normalize
x_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in x_train]) / 255.0
x_test_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in x_test]) / 255.0

# Reshape grayscale images
x_train_gray = x_train_gray[..., np.newaxis]
x_test_gray = x_test_gray[..., np.newaxis]

# Normalize RGB images
x_train = x_train / 255.0
x_test = x_test / 255.0

print(f"Training data shape: {x_train_gray.shape}, {x_train.shape}")


2025-04-05 19:16:56.584736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Training data shape: (50000, 32, 32, 1), (50000, 32, 32, 3)


In [2]:
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model

def build_generator():
    input_img = Input(shape=(32, 32, 1))

    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(3, (3, 3), padding='same')(x)
    output = Activation('tanh')(x)

    model = Model(input_img, output)
    return model


In [3]:
from tensorflow.keras.layers import Flatten, Dense

def build_discriminator():
    input_img = Input(shape=(128, 128, 3))

    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Flatten()(x)

    x = Dense(1, activation='sigmoid')(x)

    model = Model(input_img, x)
    return model


In [9]:
from tensorflow.keras.optimizers import Adam

generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer=Adam(0.0002), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

gan_input = Input(shape=(128, 128, 1))
generated_img = generator(gan_input)
validity = discriminator(generated_img)

gan = Model(gan_input, validity)
gan.compile(optimizer=Adam(0.0002), loss='binary_crossentropy')


ValueError: Input 0 of layer "functional_10" is incompatible with the layer: expected shape=(None, 32, 32, 1), found shape=(None, 128, 128)